In [1]:
import pandas as pd
import os
print('Imports done')

Imports done


In [2]:
df_taxi = pd.read_parquet('./data/green_tripdata_2025-11.parquet')

In [4]:
#Checking dtypes
df_taxi.dtypes

VendorID                          int32
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag                  str
RatecodeID                      float64
PULocationID                      int32
DOLocationID                      int32
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                       float64
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
cbd_congestion_fee              float64
dtype: object

## Question 3: 
Counting Short Trips, less or equal than a mile.

In [13]:
# Without filtering the data.
print(len(df_taxi[df_taxi['trip_distance'] <= 1]['trip_distance']))

# Filtering the data for the range [2025-11-01, 2025-12-01]
filtered_df = df_taxi[(df_taxi['trip_distance'] <= 1.0) & (df_taxi['lpep_pickup_datetime'] >= '2025-11-01') & (df_taxi['lpep_pickup_datetime'] < '2025-12-01')]
print(len(filtered_df))

8009
8007


## Question 4:
Longest trip for each day. Conditions: filter to trips that have `trip_distance` less than 100 miles.

In [18]:
df_taxi[df_taxi['trip_distance'] <= 100.00][['lpep_pickup_datetime', 'trip_distance']].sort_values(by='trip_distance', ascending=False).head(1)

,lpep_pickup_datetime,trip_distance
18867,2025-11-14 15:36:27,88.03


## Question 5:
What is the biggest pickup zone?: We need to join the zone lookup table and check the sum of total pickups per zone.

In [19]:
## Joining the table
df_taxi['PULocationID'].value_counts()

PULocationID
74     12049
75      5838
166     2182
95      2122
43      1936
       ...  
96         1
156        1
46         1
202        1
59         1
Name: count, Length: 233, dtype: int64

In [20]:
lookup_table = pd.read_csv('./data/taxi_zone_lookup.csv')
lookup_table.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [23]:
len(lookup_table['LocationID'].values)

265

In [34]:
df_joined = df_taxi.merge(lookup_table, left_on='PULocationID', right_on='LocationID', how='left')
df_joined.groupby('Zone').size().sort_values(ascending=False).head(1)

Zone
East Harlem North    12049
dtype: int64

## Question 6
Finding the largest drop off area for "East Harlem North" and the drop off area with the highest tip.

In [40]:
dict_zones = dict(zip(lookup_table['LocationID'], lookup_table['Zone']))
df_joined['PULocationZone'] = df_joined['PULocationID'].map(dict_zones)
df_joined['DOLocationZone'] = df_joined['DOLocationID'].map(dict_zones)

In [56]:
df_joined[df_joined['PULocationZone'] == 'East Harlem North'].pivot_table(values='tip_amount', index='DOLocationZone', aggfunc='max').sort_values('tip_amount', ascending=False).head(1)

,tip_amount
DOLocationZone,
Yorkville West,81.89
